In [1]:
import graphlab as gl
import matplotlib.pyplot as plt
import os
import graphlab.aggregate as agg
import numpy as np
from scipy.stats import gaussian_kde
import seaborn as sns

A newer version of GraphLab Create (v2.1) is available! Your current version is v2.0.1.
You can use pip to upgrade the graphlab-create package. For more information see https://turi.com/products/create/upgrade.


In [3]:
def Rock_and_Roll(filename): 
    #Find the edges of a group of points by Andrews Monotone Chain Convex Hull Algorithm
    def Edges(P):
        points = sorted(set(P))
        def cross(o, a, b):
            return (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0])
        lower = []
        for count in points:
                while len(lower) >= 2 and cross(lower[-2], lower[-1], count) <= 0:
                    lower.pop()
                lower.append(count)
        upper = []
        for count in reversed(points):
                while len(upper) >= 2 and cross(upper[-2], upper[-1], count) <= 0:
                    upper.pop()
                upper.append(count)
        shape = lower[:-1] + upper[:-1]
        return shape
    #Finds the Area from the edges verticies through the shoelace algorithm
    def Area(shape):
        Area = 0
        Add = 0
        Sub = 0
        for a in range(len(shape)):
            if a != (len(shape)-1):
                Add = Add + shape[a][0]* shape[a+1][1]
                Sub = Sub + shape[a][1]* shape[a+1][0]
            else:
                Add = Add + shape[a][0]* shape[0][1]
                Sub = Sub + shape[a][1]* shape[0][0]
        Area = (((Add - Sub)**2)**0.5)* 0.5
        return Area
    
    #import in the txt file as a SFrame
    Raw_Image = gl.SFrame.read_csv(filename, delimiter = '	',header = True)
    
    #shift points so the min is (0,0)
    Sauce = gl.SFrame({'X': Raw_Image['Xc']-min(Raw_Image['Xc']),'Y': Raw_Image['Yc']-min(Raw_Image['Yc'])})
    
    #Runs DBSCAN on the points
    #Cluster
    Image_C = gl.dbscan.create(Sauce, radius = 30, min_core_neighbors = 20)
    #Remove noise for % of points clustered
    Noise_Filter = gl.dbscan.create(Sauce, radius = 30, min_core_neighbors = 2)
    #Gather all points of the subject
    #Total_Area = gl.dbscan.create(Sauce, radius = 70, min_core_neighbors = 60)
    Sauce = Sauce.add_row_number('row_id')
    
    #Add the cluster identity to each point 
    Image_Points = Sauce.join(Image_C['cluster_id'], on='row_id', how='left')
    Noise_Filter_p = Sauce.join(Noise_Filter['cluster_id'], on='row_id', how='left')
    #Total_Area = Sauce.join(Total_Area['cluster_id'], on='row_id', how='left')
    
    #Removes non clustered points
    Image_Pts = Image_Points[Image_Points['cluster_id']>=0]
    Noise_Filter_p = Noise_Filter_p[Noise_Filter_p['cluster_id']>=0]
    #Total_Area = Total_Area[Total_Area['cluster_id']>=0]

    #Determines subject and treatment from filename
    if filename.find('++') > 0:
        #Treat = 'Antagonist'
        Treat = 'CytD'
    elif filename.find('+-')>0:
        #Treat = 'Nicotine'
        #Treat = 'KCl'
        Treat = 'mbCD'
    else :
        Treat = "Control"
    if filename.find('E')>0:
        Subject= 'Extension'
    elif filename.find('V')>0:
        Subject ='Varicosity' 
    else:
        Subject ='Soma'
        
    #Creates SArray to store cluster volumes in    
    Clus_Vol = gl.SArray(data=[])
    

    
    #Initialize total clustered area of subject to be 0
    Area_t = 0
    
    #Determines the areas of each cluster and total clustered area of the subject
    Image_Points = Image_Points.sort('cluster_id', ascending = False)
    print Image_Points[0]['cluster_id']
    if Image_Points[0]['cluster_id'] != None:
        #The number of clusters present in the subject
        Num_C = int((Image_Pts.sort('cluster_id', ascending = False))[0]['cluster_id'])
        for c in range(0,Num_C+1):
            cluster_mask = Image_Pts['cluster_id'] == c
            x,y = Image_Pts['X'][cluster_mask],Image_Pts['Y'][cluster_mask]
            P = zip(x,y)
            shape_c = Edges(P)
            Area_c = Area(shape_c)
            Area_t = Area_c + Area_t
            qwe = gl.SArray(data=[Area_c])
            Clus_Vol = Clus_Vol.append(qwe)
    else:
        Num_C = -1

        
    #Determines area of the total subject
    #x,y = Total_Area['X'],Total_Area['Y']
    #P = zip(x,y)
    #shape_V = Edges(P)
    #Area_Var = Area(shape_V)
    
    #Creates SFrame for Total Information by Cluster
    Total_Clusters = Image_Pts.groupby(key_columns='cluster_id',operations={'Num_points':agg.COUNT('row_id')})
    Total_Clusters = Total_Clusters.sort('cluster_id')
    Total_Clusters = Total_Clusters.add_column(Clus_Vol,name ='Cluster Area')
    Total_Clusters = Total_Clusters.add_column(Total_Clusters['Num_points']/Total_Clusters['Cluster Area'],name='Cluster Density')
    Total_Clusters['Treatment']=Treat
    Total_Clusters['Subject']=Subject
    Total_Clusters['File']=filename
    
    #Create SFrame for Total Information by Image
    Image_Data = Total_Clusters.groupby(key_columns='File', operations ={'Mean Cluster Area':agg.MEAN('Cluster Area'),
                                                                         'Mean Num Pts in Clus':agg.MEAN('Num_points'),
                                                                        'Mean Density':agg.MEAN('Cluster Density')})
    Image_Data['Treatment']=Treat
    Image_Data['Subject']=Subject
    
    Image_Data['Number of Clusters'] = Num_C+1
    #Image_Data['Area of Subject'] = Area_Var
    #Image_Data['Total Clustered Area'] = Area_t
    #Image_Data['Percent Area clusters'] = Area_t / Area_Var * 100
    Image_Data['Percent Pts clusters'] = float(Image_Pts.num_rows()) / float(Noise_Filter_p.num_rows())*100
    
   
    #cluster image
    core_mask = Image_Points['type'] == 'core'
    boundary_mask = Image_Points['type'] == 'boundary'
    noise_mask = Image_Points['type'] == 'noise'
    plt.axis([0, max(Sauce['X']), 0, max(Sauce['Y'])])
    #plt.scatter(Image_Points['X'][core_mask],Image_Points['Y'][core_mask],s=1,c=Image_Points['cluster_id'][core_mask],alpha = 0.9)
    plt.scatter(Image_Points['X'][core_mask],Image_Points['Y'][core_mask],s=1,c='red',alpha = 0.9)
    plt.scatter(Image_Points['X'][boundary_mask],Image_Points['Y'][boundary_mask],s=1,c=Image_Points['cluster_id'][boundary_mask],alpha = 0.5)
    plt.scatter(Image_Points['X'][noise_mask],Image_Points['Y'][noise_mask],s=1,c='black',alpha = 0.3)
    plt.savefig(filename + '_cluster.png', bbox_inches='tight')
    plt.close()
    
    #raw dawg
    plt.scatter(Sauce['X'],Sauce['Y'],s=1,c = 'black')
    plt.axis([0, max(Sauce['X']), 0, max(Sauce['Y'])])
    plt.savefig(filename + '_raw.png', bbox_inches='tight')
    plt.close()
    
    #Kernal density estmation image
    x = Image_Points['X']
    y = Image_Points['Y']
    xy = np.vstack([x,y])
    z = gaussian_kde(xy)(xy)
    plt.axis([0, max(Sauce['X']), 0, max(Sauce['Y'])])
    plt.scatter(x,y,s=1,c=z)
    plt.savefig(filename + '_Density.png', bbox_inches='tight')
    plt.close()
    
    #2D Histogram image
    x = Image_Points['X']
    y = Image_Points['Y']
    plt.axis([0, max(Sauce['X']), 0, max(Sauce['Y'])])
    plt.hist2d(x, y, (80, 80), cmap=plt.cm.jet)
    plt.colorbar()
    plt.savefig(filename + '_Hist.png', bbox_inches='tight')
    plt.close()
    
    
    
    
    return (Total_Clusters, Image_Data)

In [4]:
Total_Clusters = gl.SFrame({'cluster_id':gl.SArray(dtype = int),'Num_points':gl.SArray(dtype = int),
                   'Cluster Area':gl.SArray(dtype = float),
                   'Cluster Density':gl.SArray(dtype = float),'Treatment':gl.SArray(dtype = str), 'File':gl.SArray(dtype = str),
                   'Subject':gl.SArray(dtype = str)})
Total_Image =gl.SFrame({'Treatment':gl.SArray(dtype = str), 'File':gl.SArray(dtype = str),
                   'Subject':gl.SArray(dtype = str),'Number of Clusters':gl.SArray(dtype = int),
                       'Percent Pts clusters':gl.SArray(dtype = float),
                       'Mean Cluster Area':gl.SArray(dtype = float),'Mean Density':gl.SArray(dtype = float),
                       'Mean Num Pts in Clus':gl.SArray(dtype = float)})
for f in os.listdir('Cluster_Data'):     #loads up every txt file in the cluster_data dir but sources from main dir
    C,I = Rock_and_Roll(f)
    print C
    print I
    Total_Clusters = Total_Clusters.append(C) #makes master list
    Total_Image = Total_Image.append(I)  #makes master list
    Total_Clusters = Total_Clusters.sort(['Subject',"Treatment"])
    Total_Image = Total_Image.sort(['Subject',"Treatment"])
    Total_Clusters.export_csv(filename = 'Total Clusters.txt', delimiter = ' 	')
    Total_Image.export_csv(filename = 'Total Image.txt', delimiter = ' 	')

This non-commercial license of GraphLab Create for academic use is assigned to mdlycas@sund.ku.dk and will expire on March 08, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.0.1 started. Logging: C:\Users\jdr248\AppData\Local\Temp\graphlab_server_1469432356.log.0


Finished parsing file C:\Users\jdr248\130919 -- E 001.txt

Parsing completed. Parsed 100 lines in 0.0468 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[float,float,float,float,float,float,float,float,float,long,long,long,long,long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


...

| 2786                        |

| 0                           |

+-----------------------------+

[INFO] graphlab.toolkits.clustering.dbscan: Processing boundary points.


4
+------------+------------+--------------+------------------+-----------+
| cluster_id | Num_points | Cluster Area | Cluster Density  | Treatment |
+------------+------------+--------------+------------------+-----------+
|     0      |     46     |   4737.26    | 0.00971025445089 |    mbCD   |
|     1      |     40     |   2531.64    | 0.0158000347601  |    mbCD   |
|     2      |     34     |   3517.865   | 0.00966495303259 |    mbCD   |
|     3      |     23     |   1485.945   | 0.0154783656192  |    mbCD   |
|     4      |     24     |   2233.94    | 0.0107433503138  |    mbCD   |
+------------+------------+--------------+------------------+-----------+
+------------+------------------+
|  Subject   |       File       |
+------------+------------------+
| Varicosity | 140813+- V20.txt |
| Varicosity | 140813+- V20.txt |
| Varicosity | 140813+- V20.txt |
| Varicosity | 140813+- V20.txt |
| Varicosity | 140813+- V20.txt |
+------------+------------------+
[5 rows x 7 columns]

+---

In [4]:
print(plt.style.available)


[u'seaborn-darkgrid', u'seaborn-notebook', u'classic', u'seaborn-ticks', u'grayscale', u'bmh', u'seaborn-talk', u'dark_background', u'ggplot', u'fivethirtyeight', u'seaborn-colorblind', u'seaborn-deep', u'seaborn-whitegrid', u'seaborn-bright', u'seaborn-poster', u'seaborn-muted', u'seaborn-paper', u'seaborn-white', u'seaborn-pastel', u'seaborn-dark', u'seaborn-dark-palette']


In [ ]:
 #Create Images
    
 
   
    #cluster image
    core_mask = Image_Points['type'] == 'core'
    boundary_mask = Image_Points['type'] == 'boundary'
    noise_mask = Image_Points['type'] == 'noise'
    plt.axis([0, max(Sauce['X']), 0, max(Sauce['Y'])])
    #plt.scatter(Image_Points['X'][core_mask],Image_Points['Y'][core_mask],s=1,c=Image_Points['cluster_id'][core_mask],alpha = 0.9)
    plt.scatter(Image_Points['X'][core_mask],Image_Points['Y'][core_mask],s=1,c='red',alpha = 0.9)
    plt.scatter(Image_Points['X'][boundary_mask],Image_Points['Y'][boundary_mask],s=1,c=Image_Points['cluster_id'][boundary_mask],alpha = 0.5)
    plt.scatter(Image_Points['X'][noise_mask],Image_Points['Y'][noise_mask],s=1,c='black',alpha = 0.3)
    plt.savefig(filename + '_cluster.png', bbox_inches='tight')
    plt.close()
    
    #raw dawg
    plt.scatter(Sauce['X'],Sauce['Y'],s=1,c = 'black')
    plt.axis([0, max(Sauce['X']), 0, max(Sauce['Y'])])
    plt.savefig(filename + '_raw.png', bbox_inches='tight')
    plt.close()
    
    #Kernal density estmation image
    x = Image_Points['X']
    y = Image_Points['Y']
    xy = np.vstack([x,y])
    z = gaussian_kde(xy)(xy)
    plt.axis([0, max(Sauce['X']), 0, max(Sauce['Y'])])
    plt.scatter(x,y,s=1,c=z)
    plt.savefig(filename + '_Density.png', bbox_inches='tight')
    plt.close()
    
    #2D Histogram image
    x = Image_Points['X']
    y = Image_Points['Y']
    plt.axis([0, max(Sauce['X']), 0, max(Sauce['Y'])])
    plt.hist2d(x, y, (80, 80), cmap=plt.cm.jet)
    plt.colorbar()
    plt.savefig(filename + '_Hist.png', bbox_inches='tight')
    plt.close()